# Sequential Simulations with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies
Since the `OperatiotnsProblem` is the fundamental building block of a sequential
simulation in PowerSimulations, we will build on the [OperationsProblem example](../../notebook/3_PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(
    joinpath(pkgpath, "test", "3_PowerSimulations_examples", "01_operations_problems.jl"),
)

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalStandardUnitCommitment
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = FixedOutput
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
  ser

### 5-Minute system
We had already created a `sys::System` from hourly RTS data in the OperationsProblem example.
The RTS data also includes 5-minute resolution time series data. So, we can create another
`System`:

In [2]:
sys_RT = System(rawsys; time_series_resolution = Dates.Minute(5))
transform_single_time_series!(sys_RT, 12, Hour(1))

┌ Info: Unit System changed to DEVICE_BASE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/base.jl:293
┌ Warning: Missing LOAD data.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:212
┌ Warning: User-defined column name Startup Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:1364
┌ Warning: User-defined column name Shutdown Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:1364
┌ Warning: User-defined column name Status at Start is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_system_table_data.jl:1364
┌ Warning: User-defined column name Time at Status is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/qbCqK/src/parsers/power_

## `OperationsProblemTemplate`s define `Stage`s
Sequential simulations in PowerSimulations are created by defining `OperationsProblems`
that represent `Stages`, and how information flows between executions of a `Stage` and
between different `Stage`s.

Let's start by defining a two stage simulation that might look like a typical day-Ahead
and real-time electricity market clearing process.

### Define the reference model for the day-ahead unit commitment

In [3]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalStandardUnitCommitment),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :Hydro => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_uc = template_unit_commitment(devices = devices)


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalStandardUnitCommitment
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = FixedOutput
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        device_type = HVDCLine
        formulation = HVDCDispatch
  services: 
      ReserveDo

### Define the reference model for the real-time economic dispatch

In [4]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :Hydro => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_ed = template_economic_dispatch(devices = devices)


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = HydroDispatch
        formulation = FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = ThermalDispatch
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = FixedOutput
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        device_type = HVDCLine
        formulation = HVDCDispatch
  services: 


### Define the `Stage`s
Stages define models. The actual problem will change as the stage gets updated to represent
different time periods, but the formulations applied to the components is constant within
a stage. In this case, we want to define two stages with the `OperationsProblemTemplate`s
and the `System`s that we've already created.

In [5]:
stages_definition = Dict(
    "UC" => Stage(GenericOpProblem, template_uc, sys, solver),
    "ED" => Stage(
        GenericOpProblem,
        template_ed,
        sys_RT,
        solver,
        balance_slack_variables = true,
    ),
)

Dict{String,Stage{GenericOpProblem}} with 2 entries:
  "ED" => Stage()…
  "UC" => Stage()…

Note that the "ED" stage has a `balance_slack_variables = true` argument. This adds slack
variables with a default penalty of 1e6 to the nodal energy balance constraint and helps
ensure feasibility with some performance impacts.

### `SimulationSequence`
Similar to an `OperationsProblemTemplate`, the `SimulationSequence` provides a template of
how to execute a sequential set of operations problems.

print_struct(SimulationSequence)

Let's review some of the `SimulationSequence` arguments.

### Chronologies
In PowerSimulations, chronologies define where information is flowing. There are two types
of chronologies.
 - inter-stage chronologies: Define how information flows between stages. e.g. day-ahead
solutions are used to inform economic dispatch problems
 - intra-stage chronologies: Define how information flows between multiple executions of a
single stage. e.g. the dispatch setpoints of the first period of an economic dispatch problem
are constrained by the ramping limits from setpoints in the final period of the previous problem.

Let's define an inter-stage chronology that synchronizes information from 24 periods of
the first stage with a set of executions of the second stage:

In [6]:
feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

Dict{Pair{String,String},Synchronize} with 1 entry:
  "UC"=>"ED" => Synchronize(24, 0, UpdateTrigger(-1, -1))

### `FeedForward` and `Cache`
The definition of exactly what information is passed using the defined chronologies is
accomplished with `FeedForward` and `Cache` objects. Specifically, `FeedForward` is used
to define what to do with information being passed with an inter-stage chronology. Let's
define a `FeedForward` that affects the semi-continuous range constraints of thermal generators
in the economic dispatch problems based on the value of the unit-commitment variables.

In [7]:
feedforward = Dict(
    ("ED", :devices, :Generators) => SemiContinuousFF(
        binary_source_stage = PSI.ON,
        affected_variables = [PSI.ACTIVE_POWER],
    ),
)

Dict{Tuple{String,Symbol,Symbol},SemiContinuousFF} with 1 entry:
  ("ED", :devices, :Generators) => SemiContinuousFF(:On, [:P], nothing)

### Sequencing
The stage problem length, look-ahead, and other details surrounding the temporal Sequencing
of stages are controlled using the `order`, `horizons`, and `intervals` arguments.
 - order::Dict(Int, String) : the hierarchical order of stages in the simulation
 - horizons::Dict(String, Int) : defines the number of time periods in each stage (problem length)
 - intervals::Dict(String, Dates.Period) : defines the interval with which stage problems
advance after each execution. e.g. day-ahead problems have an interval of 24-hours

So, to define a typical day-ahead - real-time sequence, we can define the following:
 - Day ahead problems should represent 48 hours, advancing 24 hours after each execution (24-hour look-ahead)
 - Real time problems should represent 1 hour (12 5-minute periods), advancing 1 hour after each execution (no look-ahead)

In [8]:
order = Dict(1 => "UC", 2 => "ED")
horizons = Dict("UC" => 24, "ED" => 12)
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

Dict{String,Tuple{Hour,Consecutive}} with 2 entries:
  "ED" => (Hour(1), Consecutive(UpdateTrigger(-1, -1)))
  "UC" => (Hour(24), Consecutive(UpdateTrigger(-1, -1)))

Finally, we can put it all together:

In [9]:
DA_RT_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
)

Feed Forward Chronology
-----------------------

ED: SemiContinuousFF -> Generators

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`
Now, we can build and execute a simulation using the `SimulationSequence` and `Stage`s
that we've defined.

In [10]:
sim = Simulation(
    name = "rts-test",
    steps = 1,
    stages = stages_definition,
    stages_sequence = DA_RT_sequence,
    simulation_folder = rts_dir,
)

Simulation()


### Build simulation

In [11]:
build!(sim)

BUILT::BuildStatus = 0

### Execute simulation
the following command returns the status of the simulation (0: is proper execution) and
stores the results in a set of HDF5 files on disk.

In [12]:
execute!(sim)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 1.0694e+06 - 0.40 seconds
Cgl0003I 58 fixed, 0 tightened bounds, 1450 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 103 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1392 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1311 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1195 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 545 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 483 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 462 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 450 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 399 str

SUCCESSFUL::RunStatus = 0

## Results
To access the results, we need to load the simulation result metadata and then make
requests to the specific data of interest. This allows you to efficiently access the
results of interest without overloading resources.

In [13]:
results = SimulationResults(sim)
uc_results = get_stage_results(results, "UC"); # UC stage result metadata
ed_results = get_stage_results(results, "ED"); # ED stage result metadata

┌ Info: checking integrity of /Users/cbarrows/Documents/repos/SIIPExamples.jl/RTS-GMLC-master/rts-test/5/data_store/simulation_store.h5
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/uqxkd/src/utils.jl:380
┌ Info: Loaded time series from storage file existing=system-b45d2d52-5b1c-471b-bcbd-bcbffa5febc7_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_6cGd4t
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/Es1dj/src/hdf5_time_series_storage.jl:82
┌ Info: Loaded time series from storage file existing=system-3839d933-c2eb-414f-887c-691d2c97d945_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_rc4uZp
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/Es1dj/src/hdf5_time_series_storage.jl:82


Now we can read the specific results of interest for a specific stage, time window (optional),
and set of variables, duals, or parameters (optional)

In [14]:
read_variables(uc_results, names = [:P__ThermalStandard, :P__RenewableDispatch])

┌ Info: reading variables from data store
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/uqxkd/src/core/simulation_results.jl:212


Dict{Symbol,DataStructures.SortedDict{DateTime,DataFrame,Ord} where Ord<:Base.Order.Ordering} with 2 entries:
  :P__ThermalStandard   => DataStructures.SortedDict{DateTime,DataFrame,Base.Or…
  :P__RenewableDispatch => DataStructures.SortedDict{DateTime,DataFrame,Base.Or…

Or if we want the result of just one variable, parameter, or dual (must be defined in the
stage definition), we can use:

In [15]:
read_parameter(
    ed_results,
    :P__max_active_power__RenewableFix,
    initial_time = DateTime("2020-01-01T06:00:00"),
    count = 5,
)

┌ Info: reading parameters from data store
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/uqxkd/src/core/simulation_results.jl:295


DataStructures.SortedDict{DateTime,DataFrame,Base.Order.ForwardOrdering} with 5 entries:
  DateTime("2020-01-01T06:00:00") => 12×32 DataFrame…
  DateTime("2020-01-01T07:00:00") => 12×32 DataFrame…
  DateTime("2020-01-01T08:00:00") => 12×32 DataFrame…
  DateTime("2020-01-01T09:00:00") => 12×32 DataFrame…
  DateTime("2020-01-01T10:00:00") => 12×32 DataFrame…

* note that this returns the results of each execution step in a separate dataframe *
If you want the realized results (without lookahead periods), you can call `read_realized_*`:

In [16]:
read_realized_variables(uc_results, names = [:P__ThermalStandard, :P__RenewableDispatch])

┌ Info: reading variables from data store
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/uqxkd/src/core/simulation_results.jl:212


Dict{Symbol,DataFrame} with 2 entries:
  :P__ThermalStandard   => 24×76 DataFrame…
  :P__RenewableDispatch => 24×30 DataFrame…

## Plotting
Take a look at the examples in [the plotting folder.](../../notebook/3_PowerSimulations_examples/Plotting)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*